In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
# Inicializa el navegador
driver = webdriver.Chrome(executable_path='C:\Windows\System32\chromedriver.exe')

# Maximiza la ventana del navegador (abre en pantalla completa)
driver.maximize_window()

# Abre la página web
driver.get("https://www.autohero.com/es/search/")

# Lista para almacenar los enlaces
links = []

# Simula hacer scroll y carga los enlaces en cada scroll
for _ in range(100):  # Ajusta el número de scrolls según sea necesario
    # Hace scroll hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Espera a que la página se cargue después del scroll
    time.sleep(2)
    # Extrae los enlaces después de cada scroll y agrega a la lista
    enlaces = driver.find_elements(By.CSS_SELECTOR, 'a.link___2Maxt')
    for enlace in enlaces:
        links.append(enlace.get_attribute('href'))

# Elimina duplicados de la lista de enlaces
links = list(set(links))

# Cierra el navegador
driver.quit()

In [3]:
#Crea el dataframe con los datos de cada link
datos = pd.DataFrame()
for pagina in links[650:]:
    #si hacemos muchas consultas a la página nos echa así que vamos a poner una espera
    time.sleep(2)
    url = pagina
    r = requests.get(url)
    time.sleep(2)
    html_contents = r.text
    html_soup = BeautifulSoup(html_contents, 'html.parser')
    
    time.sleep(2)
    #Estos valores siempre aparecen en cada página
    #Vamos a sacar los elementos por su clase
    tit = html_soup.select_one('.titleText___edZSG')
    try:
        nombre = tit.text
        marca = nombre.split(' ', 1)[0]
        modelo = nombre.split(' ', 1)[1]
        precio = float(html_soup.select_one('.vehiclePrice___1uUmJ').text.strip('\xa0€').replace('.',''))
        version = html_soup.select_one('.subtitleText___2wcYx').text
        listaValores = [marca, modelo, precio, version]
        listaNombres = ['Marca', 'Modelo', 'Precio', 'Versión']

        #El resto de valores pueden o no aparecer en cada página
        #primero tratamos las etiquetas grandes de características
        carac1 = html_soup.select('.listItemValue___1IWSE')
        caracteristicas1 = [i.text for i in carac1]
        nombrecarac1 = html_soup.select('.listItemTitle___2CQBv')
        nombrecaracteristicas1 = [i.text for i in nombrecarac1]
        listaValores.extend(caracteristicas1)
        listaNombres.extend(nombrecaracteristicas1)

        #ahora se extrae la información de las tablas inferiores
        carac2 =html_soup.select('.body___2uId6')
        caracteristicas2 = [i.text for i in carac2]
        listaValores.extend(caracteristicas2)
        listaVal = [listaValores]
        nombrecarac2 =html_soup.select('.itemTitle___3GH8k')
        nombrecaracteristicas2 = [i.text for i in nombrecarac2]
        listaNombres.extend(nombrecaracteristicas2)
        datos_pagina = pd.DataFrame(listaVal,columns=listaNombres)

        datos = pd.concat([datos, datos_pagina], ignore_index=True)
    except:
        datos = datos

In [4]:
datos

,Marca,Modelo,Precio,Versión,Primera matriculación,Kilometraje,Carburante,Transmisión,Potencia,Tracción,...,Estándar de calidad,País de origen,Número de llaves,Coche accidentado y reparado,La última revisión se realizó el,Historial de revisiones,Tipo de IVA,ITV válida hasta,Matricula,Número de inventario
0,Opel,Grandland X,13399.0,1.5 CDTI Selective,23.04.2019,140.275 km,Diésel,Cambio tipo manual,131 CV / 96 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,01.05.2024,Ver historial de revisiones,IVA no deducible,30.05.2025,6675KWK,JT45070
1,Citroen,C4 Grand Picasso,10499.0,1.2 PureTech Live Edition,18.03.2016,149.604 km,Gasolina,Cambio tipo manual,130 CV / 96 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,13.06.2024,Ver historial de revisiones,IVA no deducible,22.05.2026,7697JMP,YN37235
2,Citroen,Berlingo,11799.0,1.6 Blue-HDi 20 Aniversario,31.07.2018,122.004 km,Diésel,Cambio tipo manual,99 CV / 73 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,22.03.2024,Ver historial de revisiones,IVA no deducible,31.07.2024,1783KNJ,YJ70675
3,Renault,Fluence,8299.0,1.6 Dynamique,22.06.2012,52.205 km,Gasolina,Cambio tipo manual,111 CV / 82 kW,Tracción delantera,...,Ver estándar de calidad,España,1,No,04.06.2024,Ver historial de revisiones,IVA no deducible,10.07.2024,5306HLF,VC22801
4,Hyundai,i20,8099.0,1.2 Essence,10.06.2015,97.178 km,Gasolina,Cambio tipo manual,85 CV / 62 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,04.06.2024,Ver historial de revisiones,IVA no deducible,15.06.2025,7270JGB,KF21268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,Volkswagen,Tiguan,13799.0,2.0 TDI T1 BMT,22.05.2015,132.702 km,Diésel,Cambio tipo manual,110 CV / 81 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,04.06.2024,Ver historial de revisiones,IVA no deducible,24.05.2025,2982JFP,JF14489
1300,Citroen,C-Elysée,8899.0,1.2 PureTech Shine,03.06.2019,78.891 km,Gasolina,Cambio tipo manual,82 CV / 61 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,03.06.2024,Ver historial de revisiones,IVA no deducible,09.06.2025,4080KXR,ZU60534
1301,Seat,Arona,13899.0,1.0 TSI Style,03.05.2019,27.669 km,Gasolina,Cambio tipo manual,115 CV / 85 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,15.03.2024,Ver historial de revisiones,IVA no deducible,18.05.2025,9746KWT,RL74132
1302,Nissan,Qashqai,18899.0,1.3 DIG-T N-Connecta,08.01.2020,58.870 km,Gasolina,Cambio tipo manual,140 CV / 103 kW,Tracción delantera,...,NaN,España,2,No,NaN,NaN,IVA no deducible,08.01.2026,0443LDT,GX41893


In [5]:
datos.to_excel('coches1.xlsx',index=False)